# Liquid AI LFM2.5 Interactive Demo

This notebook demonstrates how to use the Liquid AI LFM2.5-1.2B-Instruct model with an interactive widget interface.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
# Model configuration
MODEL_NAME = "LiquidAI/LFM2.5-1.2B-Instruct"

print(f"Loading model: {MODEL_NAME}")
print("This may take a few minutes...")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Define a BitsAndBytesConfig for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  # or "fp4"
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config
)

print(f"Model loaded successfully!")
print(f"Device: {next(model.parameters()).device}")

Loading model: LiquidAI/LFM2.5-1.2B-Instruct
This may take a few minutes...


Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

Model loaded successfully!
Device: cuda:0


In [3]:
def generate_answer(question: str, max_new_tokens: int = 256) -> str:
    """
    Generate an answer using the LFM2.5 model
    """
    # Create the prompt with instruction formatting
    prompt = (
        "You are a friendly teacher.\n"
        "Explain your answer step by step in simple language.\n\n"
        f"Question:\n{question}\n\nAnswer:\n"
    )
    
    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.4,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode and extract only the generated part
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = full_text[len(prompt):].strip()
    
    return answer

In [4]:
# Create interactive widgets
question_box = widgets.Textarea(
    value="Explain step by step: What is the difference between speed and velocity?",
    placeholder="Ask LFM2.5 a question (math, code, or explanation)...",
    description="Question:",
    layout=widgets.Layout(width="100%", height="120px")
)

max_tokens_slider = widgets.IntSlider(
    value=256,
    min=50,
    max=512,
    step=50,
    description="Max tokens:",
    style={'description_width': 'initial'}
)

answer_box = widgets.Textarea(
    value="",
    description="Answer:",
    layout=widgets.Layout(width="100%", height="300px")
)

button = widgets.Button(
    description="Ask LFM2.5",
    button_style="success",
    layout=widgets.Layout(width="200px")
)

status_label = widgets.HTML(value="Ready to answer questions!")

# Button click handler
def on_click(b):
    if not question_box.value.strip():
        status_label.value = "<span style='color: red;'>Please enter a question!</span>"
        return
    
    # Update status
    status_label.value = "<span style='color: blue;'>🤔 Thinking...</span>"
    answer_box.value = "Generating answer..."
    
    try:
        # Generate answer
        answer = generate_answer(question_box.value, max_tokens_slider.value)
        answer_box.value = answer
        status_label.value = "<span style='color: green;'>✅ Answer generated!</span>"
    except Exception as e:
        answer_box.value = f"Error generating answer: {str(e)}"
        status_label.value = "<span style='color: red;'>❌ Error occurred!</span>"

button.on_click(on_click)

# Display the interface
display(
    widgets.HTML("<h2>🤖 Ask LFM2.5 Anything!</h2>"),
    question_box,
    max_tokens_slider,
    widgets.HBox([button, status_label]),
    answer_box
)

HTML(value='<h2>🤖 Ask LFM2.5 Anything!</h2>')

Textarea(value='Explain step by step: What is the difference between speed and velocity?', description='Questi…

IntSlider(value=256, description='Max tokens:', max=512, min=50, step=50, style=SliderStyle(description_width=…

Textarea(value='', description='Answer:', layout=Layout(height='300px', width='100%'))

## Sample Questions to Try:

- Why is the sky blue?
- Explain the Pythagorean theorem step by step
- What is the difference between speed and velocity?
- How do Python lists work?
- Explain photosynthesis in simple terms
- What is machine learning?
- How do you solve a quadratic equation?

In [5]:
# Quick test - run this cell to test the model directly
test_question = "Why is the sky blue?"
print(f"Testing with question: {test_question}\n")
print("Answer:")
print(generate_answer(test_question))

Testing with question: Why is the sky blue?

Answer:
Let's think about this step by step.

1. First, we need to understand what color the sky appears to be.
- When the sun is shining, the sky looks blue.

2. Now, let's think about how light interacts with the atmosphere.
- Sunlight is made up of many different colors, but most of it passes through the atmosphere.

3. What happens when sunlight enters the atmosphere?
- The blue part of sunlight scatters more than the other colors.

4. Why does blue scatter more?
- Blue light has a shorter wavelength and bounces off particles in the air more than other colors.

5. What happens because of this scattering?
- The scattered blue light spreads out in all directions, making the sky appear blue to us.

6. So, the blue color we see comes from this scattering process.

Putting it all together:
The sky is blue because the blue part of sunlight scatters more in the atmosphere, and we see that scattered blue light.

Would you like to ask more questi